In [3]:
# This script replaces all previous kernel regrdding scripts. We will vertically interpolate Angie's CAM5 kernels
# to standard pressure levels, then temporally integrate to make them daily using periodic boundary conditions.
# They will also be tiled to repeat for 2 years to match the length of our CESM-LE experiments.

# By: Ty Janoski
# Updated: 03.04.2021

1. Take kernel and divide by layer thickness to produce units of W/m^2/hPa.2. Linearly interpolate to desired pressure levels with extrapolation below the surface pressure.3.  Assume each vertically-regridded monthly average kernel to the midpoint of each month, then use linear interpolation to produce daily averages.*4. Repeat step 3 for the surface pressure.5. Mask any daily kernel data below the daily surface pressure.
* using periodic boundary conditions to ensure Dec -> Jan transition is continuous.
There's a few reasons I chose to do it this way:If you do not extrapolate to the surface when you vertically regrid, you are left with NaN's in the monthly average kernels. How does one linearly interpolate between a NaN one month and a floating point number in the same location the following month? Figured it would be best to avoid that.If you linearly interpolate to daily resolution before vertically regridding, you are not only regridding the kernel values, but also the vertical levels as a consequence of the hybrid-sigma z-axis.

In [4]:
# import statements
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import ngl
import scipy.io as io
from cftime import DatetimeNoLeap

%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [ ]:
# create desired pressure levels
# because CESM output uses hybrid sigma vertical coordinates, the upper levels are isobaric surfaces
upper = np.array([  3.64346569,   7.59481965,  14.35663225,  24.61222   ,
        38.26829977,  54.59547974,  72.01245055,  87.82123029,
       103.31712663, 121.54724076, 142.99403876, 168.22507977])
# for lower level, we will go by 25 hPa
plevs = np.concatenate([upper,np.arange(200,1001,25)])

# establish correct times
times = [DatetimeNoLeap(1,1,15,12),DatetimeNoLeap(1,2,14,0),
            DatetimeNoLeap(1,3,15,12),DatetimeNoLeap(1,4,15,0),
            DatetimeNoLeap(1,5,15,12),DatetimeNoLeap(1,6,15,0),
            DatetimeNoLeap(1,7,15,12),DatetimeNoLeap(1,8,15,12),
            DatetimeNoLeap(1,9,15,0),DatetimeNoLeap(1,10,15,12),
            DatetimeNoLeap(1,11,15,0),DatetimeNoLeap(1,12,15,12)]

# create times we will interpolate to (daily)
dates = xr.cftime_range(start="0001-01-01 12:00:00",end="0001-12-31 12:00:00",freq='D',calendar='noleap')

# load normalization factor for q kernels
norm = io.loadmat('/dx05/janoski/d10/Arctic_Research/cesm-LE/output/cam5-kernels/CAM5_qkernel_normfactor.mat')['dlogqdt']
norm = norm.transpose([3,2,1,0])

# load in kernels
qkern = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/output/cam5-kernels/kernels/q.kernel.nc')
tkern = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/output/cam5-kernels/kernels/t.kernel.nc')
ps = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/output/cam5-kernels/kernels/PS.nc')

# get pressure levels from hybrid-sigma coords
hyb_p = (qkern.hyai * qkern.P0) + (qkern.hybi * ps.PS)

# calculate layer thickness from these pressure levs
pdiff = np.array(hyb_p.diff(dim='ilev').transpose('time','ilev','lat','lon'))

In [ ]:
# we are going to repeatedly use the surface pressure to mask interpolated value below the sfc
# convert to hPa, overwrite times
ps_mb = ps.PS/100
ps_mb['time'] = times

# add periodic boundary conditions
pre = ps_mb.isel(time=11)
pre['time'] = DatetimeNoLeap(0,12,15,12)

suc = ps_mb.isel(time=0)
suc['time'] = DatetimeNoLeap(2,1,15,12)

# attach boundaries as preceding december and following january
bounded = xr.concat([pre,ps_mb,suc],dim='time')

# linearly interpolate to daily timescales
ps_daily = bounded.interp(time=dates,method='linear')

In [ ]:
hold = xr.where(daily.pres < ps_daily,daily,np.nan)

In [ ]:
def interpolate(kernel,var,normalize=False, pref='t'):
    
    # the q-kernels need to be normalized to convert to dlogq
    if(normalize==True):
        dR_sigma = kernel[var]/norm
    else:
        dR_sigma = kernel[var]

    # weight by layer thickness (-> W/m^2/hPa)
    dR_weighted = dR_sigma/(pdiff/100)

    # vertically regrid to pressure levels with extrapolation to levels below the surface
    dR_p = ngl.vinth2p(dR_weighted,qkern.hyam,qkern.hybm,plevs,ps.PS,1,qkern.P0/100,1,True)

    # make our output into a data array for temporal regridding later
    da = xr.DataArray(data=dR_p,dims=['time','pres','lat','lon'],coords=dict(
        lon=(["lon"], kernel.lon),
        lat=(["lat"], kernel.lat),
        time=times,
        pres=plevs
        ),
            attrs=dict(
            units="W/m^2",
        ),
    ).rename(var)

    # add periodic boundary conditions
    pre = da.isel(time=11)
    pre['time'] = DatetimeNoLeap(0,12,15,12)

    suc = da.isel(time=0)
    suc['time'] = DatetimeNoLeap(2,1,15,12)

    # concat boundaries with kernel
    bounded = xr.concat([pre,da,suc],dim='time')

    # perform linear interpolation
    daily = bounded.interp(time=dates,method='linear')
    
    # mask values below the surface
    
    masked = xr.where(daily.pres < ps_daily,daily,np.nan).transpose('time','pres','lat','lon')
    
    # tile for two years
    tiled = np.tile(masked,(2,1,1,1))
    
    # save out kernel
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/daily_kernels/'+pref+'.kernel.'+var+'.npy'
    
    np.save(path,tiled)

In [ ]:
kern_names = [str(v) for v in qkern.data_vars][:8]
for k in kern_names:
    print(k)
    interpolate(qkern,k,normalize=True,pref='q')
kern_names = [str(v) for v in tkern.data_vars][:4]
for k in kern_names:
    print(k)
    interpolate(qkern,k,normalize=False,pref='t')

In [10]:
test = np.load('/dx05/janoski/d10/Arctic_Research/cesm-LE/daily_kernels/t.kernel.FLNT.npy')

In [6]:
test.shape

(730, 45, 192, 288)

In [14]:
plt.pcolor(test[15,-12,:,:])
plt.colorbar()
plt.title("Ty's regridded T kernel at 1000 mb")
plt.tight_layout()
plt.show()

<Figure size 432x288 with 2 Axes>

In [12]:
cmip = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/output/cam5-kernels/t.kernel.plev.nc',
                       use_cftime=True)

In [20]:
cmip.FLNT.isel(time=0,plev=3).plot()
plt.title("Angie's regridded T kernel at 1000 mb")
plt.tight_layout()
plt.show()

<Figure size 432x288 with 2 Axes>

In [ ]:
cmip.FLNT

In [19]:
cmip.plev

<xarray.DataArray 'plev' (ncl1: 17)>
array([100000,  92500,  85000,  70000,  60000,  50000,  40000,  30000,  25000,
        20000,  15000,  10000,   7000,   5000,   3000,   2000,   1000],
      dtype=int32)
Coordinates:
    plev     (ncl1) int32 100000 92500 85000 70000 60000 ... 5000 3000 2000 1000
Dimensions without coordinates: ncl1
Attributes:
    standard_name:  air_pressure
    longname:       pressure
    axis:           Z
    positive:       down
    units:          Pa

In [ ]:
plevs